# LLM을 8bit loading 하는 실험

In [1]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig
print(os.environ['TRANSFORMERS_CACHE'])

/data/huggingface-cache


## cuda 가용 메모리 체크

In [4]:
cuda_version = torch.version.cuda
free_in_GB = int(torch.cuda.mem_get_info()[0]/1024**3)
max_memory = f'{int(torch.cuda.mem_get_info()[0]/1024**3)-2}GB'
print(cuda_version, free_in_GB, max_memory)

n_gpus = torch.cuda.device_count()
max_memory = {i: max_memory for i in range(n_gpus)}
print(max_memory)

11.7 4 2GB
{0: '2GB'}


## 8bit loading

In [2]:
MAX_NEW_TOKENS = 128
model_name = "EleutherAI/gpt-neo-1.3B"

text = 'Hamburg is in which country?\n'
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
input_ids = tokenizer(text, return_tensors="pt").input_ids
print(input_ids[:10])

model = AutoModelForCausalLM.from_pretrained(
  model_name,
  device_map='auto',
  load_in_8bit=True,
  low_cpu_mem_usage=True
)

sum(p.numel() for p in model.parameters())

tensor([[  39, 4131, 3686,  318,  287,  543, 1499,   30,  198]])


Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please run

python -m bitsandbytes

 and submit this information together with your error trace to: https://github.com/TimDettmers/bitsandbytes/issues
bin /data/anaconda3/envs/ai/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so
CUDA SETUP: CUDA runtime path found: /data/anaconda3/envs/ai/lib/libcudart.so.11.0
CUDA SETUP: Highest compute capability among GPUs detected: 7.5
CUDA SETUP: Detected CUDA version 117
CUDA SETUP: Loading binary /data/anaconda3/envs/ai/lib/python3.10/site-packages/bitsandbytes/libbitsandbytes_cuda117.so...


1315575808

## llama 모델의 8bit loading

In [3]:
from accelerate import init_empty_weights
from transformers import AutoConfig, AutoModelWithLMHead

model_name = "yahma/llama-7b-hf"


model = AutoModelForCausalLM.from_pretrained(
  model_name,
  device_map='auto',
  load_in_8bit=True,
  #torch_dtype=torch.float16,
  #max_memory=max_memory,
  low_cpu_mem_usage=True
)

Overriding torch_dtype=None with `torch_dtype=torch.float16` due to requirements of `bitsandbytes` to enable model loading in mixed int8. Either pass torch_dtype=torch.float16 or don't pass this argument at all to remove this warning.


ValueError: 
                        Some modules are dispatched on the CPU or the disk. Make sure you have enough GPU RAM to fit
                        the quantized model. If you want to dispatch the model on the CPU or the disk while keeping
                        these modules in 32-bit, you need to set `load_in_8bit_fp32_cpu_offload=True` and pass a custom
                        `device_map` to `from_pretrained`. Check
                        https://huggingface.co/docs/transformers/main/en/main_classes/quantization#offload-between-cpu-and-gpu
                        for more details.
                        